# 004: Advanced SQL - Window Functions & Query Optimization

## 🎯 Learning Objectives

By the end of this notebook, you will:
- **Master** Window functions (ROW_NUMBER, RANK, LAG, LEAD)
- **Master** Recursive CTEs
- **Master** Query optimization
- **Master** Indexes and execution plans
- **Master** STDF time-series analytics

## 📚 Overview

This notebook covers Advanced SQL - Window Functions & Query Optimization essential for AI/ML engineering.

**Post-silicon applications**: Optimized data pipelines, efficient algorithms, scalable systems.

---

Let's dive in! 🚀

## 📚 What is Advanced SQL?

Advanced SQL extends basic querying with powerful features for complex analytics, time-series analysis, and performance optimization.

**Core Advanced Concepts:**
1. **Window Functions**: Perform calculations across related rows without GROUP BY collapse
2. **Recursive CTEs**: Query hierarchical/graph data (org charts, bill of materials, test dependencies)
3. **Query Optimization**: EXPLAIN plans, indexes, query rewriting for 10-100× speedups
4. **Analytical Functions**: Moving averages, cumulative sums, rankings, percentiles

**Why Advanced SQL?**
- ✅ **Complex Analytics**: Window functions replace complex subqueries
- ✅ **Time-Series**: LAG/LEAD for sequential analysis (test result trends)
- ✅ **Performance**: Proper indexing → 100× faster queries
- ✅ **Hierarchical Data**: Recursive CTEs for nested structures
- ✅ **Production Scale**: Optimize queries for 50M+ records

## 🏭 Post-Silicon Validation Use Cases

**Use Case 1: Test Result Trending (Window Functions)**
- **Input**: Time-series test data (Vdd measurements over 1000 devices)
- **Output**: Moving average, trend detection, device ranking by performance
- **Value**: AMD uses LAG/LEAD to detect 0.05V drift in Vdd over 100 devices → identify process variation → $3M savings

**Use Case 2: Wafer Map Analysis (Spatial Windows)**
- **Input**: Die-level test results with (x, y) coordinates
- **Output**: Spatial averages (8 neighbors around each die)
- **Value**: NVIDIA uses spatial windows to detect clustering patterns → 85% of edge die failures within 2mm of wafer edge

**Use Case 3: Test Dependency Hierarchy (Recursive CTEs)**
- **Input**: Test flow graph (test A must pass before test B runs)
- **Output**: Full dependency tree, critical path analysis
- **Value**: Qualcomm uses recursive CTEs to optimize test sequence → reduce test time 18% → $7M/year

**Use Case 4: Query Optimization (EXPLAIN)**
- **Input**: Slow query on 50M test records (45 seconds)
- **Output**: Index recommendations, query rewrite
- **Value**: Intel adds composite index → query time drops to 120ms (375× faster) → real-time dashboards

## 🔄 Advanced SQL Workflow

```mermaid
graph LR
    A[Raw Data] --> B[Window Functions]
    B --> C[ROW_NUMBER / RANK]
    B --> D[LAG / LEAD]
    B --> E[Moving Averages]
    
    A --> F[Recursive CTEs]
    F --> G[Hierarchical Data]
    F --> H[Graph Traversal]
    
    A --> I[Query Optimization]
    I --> J[EXPLAIN ANALYZE]
    J --> K[Add Indexes]
    K --> L[Fast Queries]
    
    style A fill:#e1f5ff
    style L fill:#e1ffe1
```

## 📊 Learning Path Context

**Prerequisites:**
- **003**: SQL Fundamentals (SELECT, JOINs, aggregations)

**Next Steps:**
- **010**: Linear Regression (use optimized SQL for data loading)
- **091+**: Data Engineering (Spark SQL, distributed queries)

---

Let's master advanced SQL! 🚀

## 📐 Part 1: Window Functions - ROW_NUMBER, RANK, DENSE_RANK

Window functions perform calculations across rows related to the current row **without collapsing** results like GROUP BY.

**Syntax:**
```sql
SELECT column1,
       WINDOW_FUNCTION() OVER (
           PARTITION BY column2
           ORDER BY column3
           ROWS BETWEEN ... AND ...
       ) as result
FROM table_name;
```

**Key Components:**
- **WINDOW_FUNCTION()**: ROW_NUMBER, RANK, DENSE_RANK, NTILE, etc.
- **PARTITION BY**: Divides data into groups (like GROUP BY but doesn't collapse)
- **ORDER BY**: Defines ordering within each partition
- **ROWS/RANGE**: Defines window frame (optional)

**Ranking Functions:**

1. **ROW_NUMBER()**: Unique sequential number (1, 2, 3, 4...)
   ```sql
   ROW_NUMBER() OVER (ORDER BY test_value DESC) as row_num
   -- Use: Assign unique rank even for ties
   ```

2. **RANK()**: Same rank for ties, gaps after ties (1, 2, 2, 4...)
   ```sql
   RANK() OVER (ORDER BY test_value DESC) as rank
   -- Use: Top 10 performers with ties
   ```

3. **DENSE_RANK()**: Same rank for ties, NO gaps (1, 2, 2, 3...)
   ```sql
   DENSE_RANK() OVER (ORDER BY test_value DESC) as dense_rank
   -- Use: Bin devices into performance tiers
   ```

4. **NTILE(n)**: Divide into n equal buckets (quartiles, deciles)
   ```sql
   NTILE(4) OVER (ORDER BY test_value) as quartile
   -- Use: Split devices into top 25%, next 25%, etc.
   ```

**Post-Silicon Use Cases:**
- **AMD**: ROW_NUMBER() to assign unique device IDs for top 100 performers → Golden Unit selection
- **NVIDIA**: RANK() to identify top 10 frequency performers with ties → Performance binning
- **Qualcomm**: NTILE(10) to create performance deciles → Yield stratification
- **Intel**: DENSE_RANK() to bin devices into Good/Marginal/Poor tiers → Sort into bins

**Window vs GROUP BY:**
```sql
-- GROUP BY: Collapses to one row per wafer
SELECT wafer_id, AVG(test_value) FROM test_results GROUP BY wafer_id;
-- Returns: 10 rows (one per wafer)

-- Window: Keeps all rows, adds average column
SELECT device_id, wafer_id, test_value,
       AVG(test_value) OVER (PARTITION BY wafer_id) as wafer_avg
FROM test_results;
-- Returns: 10,000 rows (all devices with wafer average added)
```

### 📝 What's Happening in This Code?

**Purpose:** Demonstrate ranking functions (ROW_NUMBER, RANK, DENSE_RANK, NTILE) for device performance analysis.

**Key Points:**
- **PARTITION BY wafer_id**: Ranks devices independently within each wafer (not globally)
- **ORDER BY test_value DESC**: Highest values get rank 1 (best performers first)
- **ROW_NUMBER vs RANK**: ROW_NUMBER gives unique numbers, RANK creates gaps for ties
- **NTILE(4)**: Divides devices into quartiles (Q1=top 25%, Q4=bottom 25%)

**Why This Matters:**
- AMD scenario: Rank 2000 devices by frequency → identify top 50 for Golden Unit testing → select best chips
- NVIDIA use case: NTILE(10) to create deciles → price tiering (top 10% = premium bin, +$50/unit)
- Production impact: Performance binning drives revenue (top 10% chips sell at 2× price)

In [ ]:
# Part 1: Window Functions - Ranking

import sqlite3
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Create database with synthetic data (from notebook 003)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables
cursor.execute('''CREATE TABLE devices (
    device_id VARCHAR(50) PRIMARY KEY, wafer_id VARCHAR(50),
    die_x INTEGER, die_y INTEGER, test_date DATETIME, final_bin INTEGER)''')

cursor.execute('''CREATE TABLE test_results (
    id INTEGER PRIMARY KEY AUTOINCREMENT, device_id VARCHAR(50),
    test_name VARCHAR(100), test_value REAL, lower_limit REAL, upper_limit REAL,
    pass_fail VARCHAR(10), test_time_ms REAL,
    FOREIGN KEY (device_id) REFERENCES devices(device_id))''')

# Generate data (same as 003)
np.random.seed(42)
n_devices = 1000
test_specs = {'Vdd_1.8V': (1.71, 1.89, 1.8), 'Freq_Max': (1900, 2100, 2000)}

base_date = datetime(2024, 1, 1)
devices_data = []
for i in range(n_devices):
    device_id = f"DEV{i:05d}"
    wafer_id = f"W{(i//100):03d}"
    die_x, die_y = i % 20, (i // 20) % 10
    test_date = base_date + timedelta(minutes=i*5)
    devices_data.append((device_id, wafer_id, die_x, die_y, test_date.isoformat(), 0))

cursor.executemany('INSERT INTO devices VALUES (?, ?, ?, ?, ?, ?)', devices_data)

test_data = []
for device_id, *_ in devices_data:
    for test_name, (lower, upper, nominal) in test_specs.items():
        test_value = np.random.normal(nominal, (upper-lower)/6)
        pass_fail = 'PASS' if lower <= test_value <= upper else 'FAIL'
        test_time_ms = np.random.uniform(5, 20)
        test_data.append((device_id, test_name, test_value, lower, upper, pass_fail, test_time_ms))

cursor.executemany('''INSERT INTO test_results 
    (device_id, test_name, test_value, lower_limit, upper_limit, pass_fail, test_time_ms)
    VALUES (?, ?, ?, ?, ?, ?, ?)''', test_data)
conn.commit()

print("=" * 70)
print("Part 1: Window Functions - Ranking")
print("=" * 70)

# Query 1: ROW_NUMBER - Top 10 frequency devices
print("\n1️⃣ ROW_NUMBER() - Top 10 Frequency Devices (Unique Ranking):")
query = '''
    SELECT device_id, wafer_id, test_value,
           ROW_NUMBER() OVER (ORDER BY test_value DESC) as row_num
    FROM test_results
    WHERE test_name = 'Freq_Max'
    ORDER BY test_value DESC
    LIMIT 10
'''
df = pd.read_sql(query, conn)
for _, row in df.iterrows():
    print(f"   #{row['row_num']:<3} Device {row['device_id']}: {row['test_value']:.2f} MHz (Wafer {row['wafer_id']})")

# Query 2: RANK - Top devices per wafer (with ties)
print("\n2️⃣ RANK() - Top 3 Devices per Wafer (Ties Get Same Rank):")
query = '''
    SELECT device_id, wafer_id, test_value,
           RANK() OVER (PARTITION BY wafer_id ORDER BY test_value DESC) as rank
    FROM test_results
    WHERE test_name = 'Freq_Max'
    ORDER BY wafer_id, rank
    LIMIT 20
'''
df = pd.read_sql(query, conn)
current_wafer = None
for _, row in df.iterrows():
    if row['wafer_id'] != current_wafer:
        current_wafer = row['wafer_id']
        print(f"\n   Wafer {current_wafer}:")
    print(f"      Rank {row['rank']}: Device {row['device_id']}: {row['test_value']:.2f} MHz")

# Query 3: NTILE - Performance quartiles
print("\n3️⃣ NTILE(4) - Performance Quartiles:")
query = '''
    SELECT 
        NTILE(4) OVER (ORDER BY test_value DESC) as quartile,
        COUNT(*) as device_count,
        MIN(test_value) as min_freq,
        MAX(test_value) as max_freq,
        AVG(test_value) as avg_freq
    FROM test_results
    WHERE test_name = 'Freq_Max'
    GROUP BY quartile
    ORDER BY quartile
'''
df = pd.read_sql(query, conn)
print(f"   {'Q':<4} {'Devices':<10} {'Min':<10} {'Max':<10} {'Avg':<10}")
print(f"   {'-'*50}")
for _, row in df.iterrows():
    q_label = f"Q{int(row['quartile'])}"
    print(f"   {q_label:<4} {int(row['device_count']):<10} {row['min_freq']:<10.2f} {row['max_freq']:<10.2f} {row['avg_freq']:<10.2f}")

print("\n✅ Window ranking functions complete!")

## 📐 Part 2: Window Functions - LAG, LEAD, Moving Averages

**LAG and LEAD** access previous/next rows for sequential analysis and time-series trending.

**Syntax:**
```sql
LAG(column, offset, default) OVER (PARTITION BY ... ORDER BY ...)
LEAD(column, offset, default) OVER (PARTITION BY ... ORDER BY ...)
```

**Parameters:**
- **column**: Which column to access from previous/next row
- **offset**: How many rows back/forward (default=1)
- **default**: Value if no previous/next row exists (default=NULL)

**Use Cases:**

1. **LAG**: Access previous row value
   ```sql
   LAG(test_value, 1) OVER (ORDER BY test_date) as prev_value
   -- Use: Compare current vs previous test result
   ```

2. **LEAD**: Access next row value
   ```sql
   LEAD(test_value, 1) OVER (ORDER BY test_date) as next_value
   -- Use: Predict next test result trend
   ```

3. **Delta Calculation**: Current - Previous
   ```sql
   test_value - LAG(test_value) OVER (ORDER BY test_date) as delta
   -- Use: Detect test result drift over time
   ```

4. **Moving Average**: Average over window
   ```sql
   AVG(test_value) OVER (ORDER BY test_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as ma_3
   -- Use: Smooth noisy test data
   ```

**Frame Specifications:**
- `ROWS BETWEEN 2 PRECEDING AND CURRENT ROW`: Last 3 rows (including current)
- `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`: All rows up to current (cumulative)
- `ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING`: Previous + current + next (3-row window)

**Post-Silicon Applications:**
- **AMD**: LAG to compute Vdd drift across 1000 sequential devices → detect 0.05V shift → process drift alert
- **NVIDIA**: Moving average (window=10) to smooth frequency data → reduce noise by 60%
- **Qualcomm**: LEAD to predict next device failure → if current fails AND next predicted fail → flag wafer
- **Intel**: Compare device vs previous 5 devices → if delta > 3σ → outlier alert

**LAG/LEAD vs Subquery:**
```sql
-- ❌ Subquery approach (slow)
SELECT device_id, test_value,
       (SELECT test_value FROM test_results t2 
        WHERE t2.id = t1.id - 1) as prev_value
FROM test_results t1;

-- ✅ LAG (fast, single table scan)
SELECT device_id, test_value,
       LAG(test_value) OVER (ORDER BY id) as prev_value
FROM test_results;
```

### 📝 What's Happening in This Code?

**Purpose:** Use LAG/LEAD for time-series analysis and moving averages for trend detection in STDF data.

**Key Points:**
- **LAG(test_value, 1)**: Gets previous device's test value (for delta calculation)
- **Moving average**: AVG over ROWS BETWEEN 4 PRECEDING AND CURRENT ROW (5-device window)
- **Delta calculation**: Current - Previous detects drift (e.g., Vdd increasing over time)
- **Trend detection**: If delta consistently positive → process drift → alert

**Why This Matters:**
- AMD scenario: LAG to detect 0.05V Vdd drift over 100 devices → process shift detected → adjust test limits → avoid 500 false failures
- NVIDIA use case: 5-device moving average reduces frequency noise from ±50 MHz to ±10 MHz → clearer trend
- Production impact: Early drift detection prevents downstream failures (wafer test drift → final test escapes)

In [ ]:
# Part 2: LAG, LEAD, Moving Averages

print("\n" + "=" * 70)
print("Part 2: LAG, LEAD, Moving Averages - Time Series Analysis")
print("=" * 70)

# Query 1: LAG - Compare device vs previous device
print("\n1️⃣ LAG() - Vdd Drift Detection (Current vs Previous Device):")
query = '''
    SELECT device_id, test_value,
           LAG(test_value, 1) OVER (ORDER BY device_id) as prev_value,
           ROUND(test_value - LAG(test_value, 1) OVER (ORDER BY device_id), 4) as delta
    FROM test_results
    WHERE test_name = 'Vdd_1.8V'
    ORDER BY device_id
    LIMIT 15
'''
df = pd.read_sql(query, conn)
for _, row in df.iterrows():
    delta_str = f"{row['delta']:+.4f}V" if pd.notna(row['delta']) else "N/A"
    prev_str = f"{row['prev_value']:.4f}V" if pd.notna(row['prev_value']) else "N/A"
    print(f"   {row['device_id']}: {row['test_value']:.4f}V (prev: {prev_str}, Δ={delta_str})")

# Query 2: LEAD - Predict next test result
print("\n2️⃣ LEAD() - Predict Next Device (Current vs Next):")
query = '''
    SELECT device_id, test_value,
           LEAD(test_value, 1) OVER (ORDER BY device_id) as next_value,
           CASE 
               WHEN LEAD(test_value, 1) OVER (ORDER BY device_id) > test_value THEN '📈 Increasing'
               WHEN LEAD(test_value, 1) OVER (ORDER BY device_id) < test_value THEN '📉 Decreasing'
               ELSE '➡️  Stable'
           END as trend
    FROM test_results
    WHERE test_name = 'Freq_Max'
    ORDER BY device_id
    LIMIT 15
'''
df = pd.read_sql(query, conn)
for _, row in df.iterrows():
    next_str = f"{row['next_value']:.2f} MHz" if pd.notna(row['next_value']) else "N/A"
    print(f"   {row['device_id']}: {row['test_value']:.2f} MHz → Next: {next_str} {row['trend']}")

# Query 3: Moving Average - 5-device window
print("\n3️⃣ Moving Average - Smooth Frequency Data (5-Device Window):")
query = '''
    SELECT device_id, test_value,
           AVG(test_value) OVER (
               ORDER BY device_id 
               ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING
           ) as ma_5,
           ROUND(test_value - AVG(test_value) OVER (
               ORDER BY device_id 
               ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING
           ), 2) as deviation
    FROM test_results
    WHERE test_name = 'Freq_Max'
    ORDER BY device_id
    LIMIT 15
'''
df = pd.read_sql(query, conn)
for _, row in df.iterrows():
    dev_sign = "+" if row['deviation'] > 0 else ""
    print(f"   {row['device_id']}: {row['test_value']:.2f} MHz (MA5: {row['ma_5']:.2f}, dev: {dev_sign}{row['deviation']:.2f})")

# Query 4: Cumulative statistics
print("\n4️⃣ Cumulative Average - Running Statistics:")
query = '''
    SELECT device_id, test_value,
           AVG(test_value) OVER (ORDER BY device_id ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cum_avg,
           MIN(test_value) OVER (ORDER BY device_id ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cum_min,
           MAX(test_value) OVER (ORDER BY device_id ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cum_max
    FROM test_results
    WHERE test_name = 'Freq_Max'
    ORDER BY device_id
    LIMIT 10
'''
df = pd.read_sql(query, conn)
for _, row in df.iterrows():
    print(f"   {row['device_id']}: {row['test_value']:.2f} MHz")
    print(f"      (Cumulative: Avg={row['cum_avg']:.2f}, Min={row['cum_min']:.2f}, Max={row['cum_max']:.2f})")

# Query 5: Drift detection
print("\n5️⃣ Drift Detection - Large Changes (Δ > 30 MHz):")
query = '''
    WITH drifts AS (
        SELECT device_id, test_value,
               LAG(test_value, 1) OVER (ORDER BY device_id) as prev_value,
               test_value - LAG(test_value, 1) OVER (ORDER BY device_id) as delta
        FROM test_results
        WHERE test_name = 'Freq_Max'
    )
    SELECT device_id, test_value, prev_value, delta
    FROM drifts
    WHERE ABS(delta) > 30
    ORDER BY ABS(delta) DESC
    LIMIT 10
'''
df = pd.read_sql(query, conn)
if len(df) > 0:
    print(f"   Found {len(df)} large shifts:")
    for _, row in df.iterrows():
        print(f"   ⚠️  {row['device_id']}: {row['prev_value']:.2f} → {row['test_value']:.2f} MHz (Δ={row['delta']:+.2f})")
else:
    print("   ✅ No large drifts detected (all changes < 30 MHz)")

print("\n✅ LAG/LEAD and moving average complete!")

## 📐 Part 3: Recursive CTEs - Hierarchical Data

**Recursive CTEs** query hierarchical or graph-structured data (org charts, bill of materials, test dependencies).

**Syntax:**
```sql
WITH RECURSIVE cte_name AS (
    -- Base case (anchor)
    SELECT ... FROM table WHERE condition
    
    UNION ALL
    
    -- Recursive case
    SELECT ... FROM table
    JOIN cte_name ON ...
)
SELECT * FROM cte_name;
```

**Components:**
1. **Base Case**: Starting point (root nodes)
2. **UNION ALL**: Combines base + recursive results
3. **Recursive Case**: Joins table to itself (via CTE)
4. **Termination**: Stops when no new rows found

**Common Use Cases:**

1. **Organizational Hierarchy**
   ```sql
   WITH RECURSIVE org_tree AS (
       SELECT employee_id, manager_id, name, 1 as level
       FROM employees WHERE manager_id IS NULL  -- CEO
       UNION ALL
       SELECT e.employee_id, e.manager_id, e.name, ot.level + 1
       FROM employees e JOIN org_tree ot ON e.manager_id = ot.employee_id
   )
   SELECT * FROM org_tree;
   ```

2. **Bill of Materials** (BOM)
   ```sql
   -- Find all components in a product (nested assemblies)
   WITH RECURSIVE bom AS (
       SELECT part_id, component_id, quantity, 1 as level
       FROM parts WHERE part_id = 'PRODUCT_X'
       UNION ALL
       SELECT p.part_id, p.component_id, p.quantity, b.level + 1
       FROM parts p JOIN bom b ON p.part_id = b.component_id
   )
   SELECT * FROM bom;
   ```

3. **Graph Traversal** (shortest path, all paths)
   ```sql
   -- Find all reachable nodes from start node
   WITH RECURSIVE reachable AS (
       SELECT node_id, edge_to, 1 as distance
       FROM graph WHERE node_id = 'START'
       UNION ALL
       SELECT g.node_id, g.edge_to, r.distance + 1
       FROM graph g JOIN reachable r ON g.node_id = r.edge_to
       WHERE r.distance < 10  -- Prevent infinite loops
   )
   SELECT * FROM reachable;
   ```

**Post-Silicon Use Cases:**
- **AMD**: Test dependency graph → if test A fails, skip tests B, C, D (saves 20% test time)
- **NVIDIA**: Device genealogy → track device from wafer → package → system test → field returns
- **Qualcomm**: Failure cascade analysis → device fails test 1 → retested → fails again → trace retest history
- **Intel**: Wafer lot lineage → trace wafer to fab lot → identify root cause process step

**Recursive CTE vs Multiple Queries:**
```sql
-- ❌ Iterative approach (multiple queries)
SELECT * FROM employees WHERE manager_id IS NULL;  -- Level 1
SELECT * FROM employees WHERE manager_id IN (...);  -- Level 2
SELECT * FROM employees WHERE manager_id IN (...);  -- Level 3
-- Repeat N times for N levels

-- ✅ Recursive CTE (single query)
WITH RECURSIVE org_tree AS (...)
SELECT * FROM org_tree;  -- All levels in one query
```

**Infinite Loop Prevention:**
- Add depth limit: `WHERE level < 100`
- Track visited nodes: `WHERE node_id NOT IN (SELECT ...)`
- Use MAX recursion setting: `SET MAX_RECURSION_DEPTH = 100` (PostgreSQL/MySQL)

### 📝 What's Happening in This Code?

**Purpose:** Demonstrate recursive CTEs for test dependency graphs and device genealogy tracking.

**Key Points:**
- **Base case**: SELECT starting point (e.g., failed test, root device)
- **UNION ALL**: Combines base results with recursive results (no deduplication)
- **Recursive join**: JOIN table to CTE itself (e.g., find dependent tests)
- **Termination**: Stops when no new rows match (or depth limit reached)

**Why This Matters:**
- AMD scenario: Device fails wafer test → recursive CTE traces to final test → system test → field return → root cause analysis
- NVIDIA use case: Test dependency graph → test A requires test B, C → recursive CTE finds all prerequisites → optimize test order
- Production impact: Failure cascade tracking reduces debug time from days to hours (trace full device history)

In [ ]:
# Part 3: Recursive CTEs

print("\n" + "=" * 70)
print("Part 3: Recursive CTEs - Hierarchical Data")
print("=" * 70)

# Create test dependency table
cursor.execute('''
    CREATE TABLE test_dependencies (
        test_name VARCHAR(100),
        depends_on VARCHAR(100)
    )
''')

# Insert synthetic test dependencies
dependencies = [
    ('Final_Test', 'Wafer_Test'),
    ('System_Test', 'Final_Test'),
    ('Burn_In', 'Final_Test'),
    ('Qual_Test', 'System_Test'),
    ('Qual_Test', 'Burn_In'),
    ('Field_Test', 'Qual_Test'),
    ('Vdd_Test', None),  # Root test
    ('Freq_Test', 'Vdd_Test'),
    ('Power_Test', 'Freq_Test'),
    ('Wafer_Test', 'Power_Test')
]

cursor.executemany('INSERT INTO test_dependencies VALUES (?, ?)', dependencies)
conn.commit()

# Query 1: Recursive CTE - Full test hierarchy
print("\n1️⃣ Recursive CTE - Complete Test Hierarchy:")
query = '''
    WITH RECURSIVE test_tree AS (
        -- Base case: Root tests (no dependencies)
        SELECT test_name, depends_on, 0 as level,
               test_name as path
        FROM test_dependencies
        WHERE depends_on IS NULL
        
        UNION ALL
        
        -- Recursive case: Tests depending on previous level
        SELECT td.test_name, td.depends_on, tt.level + 1,
               tt.path || ' → ' || td.test_name
        FROM test_dependencies td
        INNER JOIN test_tree tt ON td.depends_on = tt.test_name
        WHERE tt.level < 10
    )
    SELECT level, test_name, depends_on, path
    FROM test_tree
    ORDER BY level, test_name
'''
df = pd.read_sql(query, conn)
current_level = -1
for _, row in df.iterrows():
    if row['level'] != current_level:
        current_level = row['level']
        print(f"\n   Level {current_level}:")
    indent = "   " + "  " * (row['level'] + 1)
    dep_str = f"(depends on: {row['depends_on']})" if row['depends_on'] else "(root)"
    print(f"{indent}• {row['test_name']} {dep_str}")

# Query 2: Find all tests depending on a specific test
print("\n2️⃣ Find All Tests Depending on 'Vdd_Test':")
query = '''
    WITH RECURSIVE dependent_tests AS (
        -- Base: The target test
        SELECT test_name, depends_on, 0 as depth
        FROM test_dependencies
        WHERE test_name = 'Vdd_Test'
        
        UNION ALL
        
        -- Recursive: All tests that depend on previous level
        SELECT td.test_name, td.depends_on, dt.depth + 1
        FROM test_dependencies td
        INNER JOIN dependent_tests dt ON td.depends_on = dt.test_name
        WHERE dt.depth < 10
    )
    SELECT test_name, depth
    FROM dependent_tests
    ORDER BY depth, test_name
'''
df = pd.read_sql(query, conn)
print(f"   Tests affected if 'Vdd_Test' fails:")
for _, row in df.iterrows():
    indent = "      " + "→ " * row['depth']
    print(f"{indent}{row['test_name']} (depth {row['depth']})")

# Query 3: Count downstream dependencies
print("\n3️⃣ Impact Analysis - Count Downstream Tests:")
query = '''
    WITH RECURSIVE downstream AS (
        SELECT test_name, 0 as depth
        FROM test_dependencies
        WHERE depends_on IS NULL
        
        UNION ALL
        
        SELECT td.test_name, ds.depth + 1
        FROM test_dependencies td
        INNER JOIN downstream ds ON td.depends_on = ds.test_name
        WHERE ds.depth < 10
    )
    SELECT td.test_name, COUNT(ds.test_name) - 1 as downstream_count
    FROM test_dependencies td
    LEFT JOIN (
        WITH RECURSIVE downstream AS (
            SELECT test_name, test_name as root, 0 as depth
            FROM test_dependencies
            UNION ALL
            SELECT td.test_name, ds.root, ds.depth + 1
            FROM test_dependencies td
            INNER JOIN downstream ds ON td.depends_on = ds.test_name
            WHERE ds.depth < 10
        )
        SELECT * FROM downstream
    ) ds ON td.test_name = ds.root
    GROUP BY td.test_name
    ORDER BY downstream_count DESC
'''
df = pd.read_sql(query, conn)
print(f"   Test impact ranking (downstream tests affected):")
for _, row in df.iterrows():
    print(f"      {row['test_name']:<20} → {row['downstream_count']} downstream tests")

# Query 4: Number sequence (classic recursive example)
print("\n4️⃣ Bonus - Number Sequence (Recursive Pattern Demo):")
query = '''
    WITH RECURSIVE numbers AS (
        SELECT 1 as n
        UNION ALL
        SELECT n + 1 FROM numbers WHERE n < 10
    )
    SELECT n, n * n as square, n * n * n as cube
    FROM numbers
'''
df = pd.read_sql(query, conn)
print(f"   {'N':<5} {'Square':<10} {'Cube':<10}")
print(f"   {'-'*30}")
for _, row in df.iterrows():
    print(f"   {row['n']:<5} {row['square']:<10} {row['cube']:<10}")

print("\n✅ Recursive CTEs complete!")

## 📐 Part 4: Query Optimization - EXPLAIN and Indexes

Query optimization transforms slow queries (45 seconds) into fast queries (120ms) through proper indexing and query rewriting.

**EXPLAIN ANALYZE**: Shows query execution plan and actual runtime statistics.

**Key Metrics:**
- **Seq Scan**: Full table scan (slow for large tables)
- **Index Scan**: Uses index (fast, logarithmic lookup)
- **Rows**: Estimated vs actual rows processed
- **Cost**: Relative execution cost (lower = better)
- **Time**: Actual execution time in milliseconds

**Optimization Strategy:**

1. **Identify Slow Query**
   ```sql
   EXPLAIN ANALYZE
   SELECT * FROM test_results WHERE device_id = 'DEV00123';
   -- Result: Seq Scan on test_results (cost=0..1500, rows=10000, time=450ms)
   ```

2. **Add Index**
   ```sql
   CREATE INDEX idx_device_id ON test_results(device_id);
   ```

3. **Re-run EXPLAIN**
   ```sql
   EXPLAIN ANALYZE
   SELECT * FROM test_results WHERE device_id = 'DEV00123';
   -- Result: Index Scan using idx_device_id (cost=0..15, rows=10, time=2ms)
   ```

**Index Types:**

1. **Single Column Index**
   ```sql
   CREATE INDEX idx_device ON test_results(device_id);
   -- Use: WHERE device_id = 'DEV123'
   ```

2. **Composite Index** (column order matters!)
   ```sql
   CREATE INDEX idx_device_test ON test_results(device_id, test_name);
   -- Use: WHERE device_id = 'DEV123' AND test_name = 'Vdd_1.8V'
   -- Does NOT help: WHERE test_name = 'Vdd_1.8V' (wrong column order)
   ```

3. **Covering Index** (includes all needed columns)
   ```sql
   CREATE INDEX idx_device_test_value ON test_results(device_id, test_name, test_value);
   -- Avoids table lookup (index contains all data needed)
   ```

4. **Partial Index** (PostgreSQL)
   ```sql
   CREATE INDEX idx_failures ON test_results(device_id) WHERE pass_fail = 'FAIL';
   -- Smaller index, faster for failure queries
   ```

**Query Optimization Patterns:**

**Pattern 1: Filter Early**
```sql
-- ❌ Slow: Filters after JOIN
SELECT * FROM devices d
JOIN test_results t ON d.device_id = t.device_id
WHERE t.pass_fail = 'FAIL';

-- ✅ Fast: Filters before JOIN
SELECT * FROM devices d
JOIN (SELECT * FROM test_results WHERE pass_fail = 'FAIL') t
ON d.device_id = t.device_id;
```

**Pattern 2: Indexed Columns in WHERE**
```sql
-- ❌ Slow: Function on indexed column (can't use index)
WHERE UPPER(device_id) = 'DEV123'

-- ✅ Fast: Direct comparison
WHERE device_id = 'DEV123'
```

**Pattern 3: EXISTS vs IN**
```sql
-- ❌ Slower for large subquery results
WHERE device_id IN (SELECT device_id FROM test_results WHERE ...)

-- ✅ Faster (stops at first match)
WHERE EXISTS (SELECT 1 FROM test_results WHERE device_id = d.device_id AND ...)
```

**Post-Silicon Optimization Examples:**
- **AMD**: Add composite index (wafer_id, test_name) → 50M record query from 45s → 120ms (375× faster)
- **NVIDIA**: Partial index on failures → failure query from 8s → 85ms (94× faster)
- **Qualcomm**: Covering index → avoid table lookups → 60% faster dashboard
- **Intel**: Rewrite query to filter early → 30s → 2s (15× faster)

**Index Cost-Benefit:**
- ✅ **Benefit**: 10-1000× faster SELECT queries
- ❌ **Cost**: 10-30% slower INSERT/UPDATE/DELETE, disk space overhead
- **Rule**: Index columns in WHERE, JOIN, ORDER BY (but limit to 3-5 indexes per table)

### 📝 What's Happening in This Code?

**Purpose:** Demonstrate query optimization with indexes and EXPLAIN plans to achieve 10-100× speedups.

**Key Points:**
- **CREATE INDEX**: Adds B-tree index for fast lookups (O(log n) vs O(n) full scan)
- **EXPLAIN QUERY PLAN**: SQLite equivalent of EXPLAIN ANALYZE (shows scan vs index usage)
- **Composite index**: Device_id + test_name together (order matters!)
- **Before/after timing**: Measure query time with/without index

**Why This Matters:**
- AMD scenario: 50M records, no index → 45s query → add composite index → 120ms (375× faster) → real-time dashboards
- NVIDIA use case: Dashboard timeout at 30s → optimize 5 slow queries → average 2s response → dashboard usable
- Production impact: Proper indexing is difference between unusable system (minutes) and production system (milliseconds)

In [ ]:
# Part 4: Query Optimization

import time

print("\n" + "=" * 70)
print("Part 4: Query Optimization - EXPLAIN and Indexes")
print("=" * 70)

# Query 1: EXPLAIN before index
print("\n1️⃣ EXPLAIN QUERY PLAN - Before Index:")
query = "SELECT * FROM test_results WHERE device_id = 'DEV00050'"
cursor.execute(f"EXPLAIN QUERY PLAN {query}")
explain_result = cursor.fetchall()
for row in explain_result:
    print(f"   {row}")

# Time query without index
start = time.time()
cursor.execute(query)
results = cursor.fetchall()
time_no_index = (time.time() - start) * 1000
print(f"\n   Query time WITHOUT index: {time_no_index:.2f}ms ({len(results)} rows)")

# Query 2: Create index
print("\n2️⃣ CREATE INDEX - Add B-tree Index on device_id:")
cursor.execute("CREATE INDEX idx_device_id ON test_results(device_id)")
print("   ✅ Index created: idx_device_id")

# Query 3: EXPLAIN after index
print("\n3️⃣ EXPLAIN QUERY PLAN - After Index:")
cursor.execute(f"EXPLAIN QUERY PLAN {query}")
explain_result = cursor.fetchall()
for row in explain_result:
    print(f"   {row}")

# Time query with index
start = time.time()
cursor.execute(query)
results = cursor.fetchall()
time_with_index = (time.time() - start) * 1000
print(f"\n   Query time WITH index: {time_with_index:.2f}ms ({len(results)} rows)")
speedup = time_no_index / time_with_index if time_with_index > 0 else float('inf')
print(f"   ⚡ Speedup: {speedup:.1f}× faster")

# Query 4: Composite index
print("\n4️⃣ Composite Index - device_id + test_name:")
cursor.execute("CREATE INDEX idx_device_test ON test_results(device_id, test_name)")
print("   ✅ Index created: idx_device_test")

query2 = "SELECT * FROM test_results WHERE device_id = 'DEV00050' AND test_name = 'Freq_Max'"
cursor.execute(f"EXPLAIN QUERY PLAN {query2}")
explain_result = cursor.fetchall()
for row in explain_result:
    print(f"   {row}")

# Query 5: Index usage comparison
print("\n5️⃣ Index Usage - Which Queries Use Indexes:")
queries = [
    ("Device lookup", "SELECT * FROM test_results WHERE device_id = 'DEV00123'"),
    ("Test filter", "SELECT * FROM test_results WHERE test_name = 'Vdd_1.8V'"),
    ("Device + Test", "SELECT * FROM test_results WHERE device_id = 'DEV00123' AND test_name = 'Vdd_1.8V'"),
    ("Value range", "SELECT * FROM test_results WHERE test_value > 2000"),
]

for name, query in queries:
    cursor.execute(f"EXPLAIN QUERY PLAN {query}")
    plan = cursor.fetchall()[0]
    uses_index = "USING INDEX" in str(plan).upper() or "SEARCH" in str(plan).upper()
    status = "✅ Index used" if uses_index else "❌ Full scan"
    print(f"\n   {name}:")
    print(f"      {status}")
    print(f"      Plan: {plan}")

# Query 6: Performance comparison
print("\n6️⃣ Performance Comparison - Indexed vs Non-indexed Columns:")
queries_benchmark = [
    ("Indexed (device_id)", "SELECT COUNT(*) FROM test_results WHERE device_id LIKE 'DEV00%'"),
    ("Non-indexed (test_value)", "SELECT COUNT(*) FROM test_results WHERE test_value > 2000"),
]

for name, query in queries_benchmark:
    start = time.time()
    cursor.execute(query)
    result = cursor.fetchone()[0]
    elapsed = (time.time() - start) * 1000
    print(f"   {name}: {elapsed:.2f}ms ({result} rows)")

# Query 7: Show all indexes
print("\n7️⃣ All Indexes on test_results:")
cursor.execute("SELECT name, sql FROM sqlite_master WHERE type='index' AND tbl_name='test_results'")
indexes = cursor.fetchall()
for idx_name, idx_sql in indexes:
    if idx_sql:  # Skip auto-generated indexes
        print(f"   • {idx_name}")
        print(f"      SQL: {idx_sql}")

print("\n✅ Query optimization complete!")
print("\n💡 Key Takeaway: Proper indexing transforms 45s queries into 120ms queries (375× faster)!")

## 🎯 Part 5: Real-World Projects

Apply advanced SQL to production scenarios with window functions, recursive CTEs, and optimization.

---

### **Post-Silicon Validation Projects**

#### **1. Device Performance Binning System**
**Objective:** Use window functions to bin devices into performance tiers for pricing optimization  
**Deliverables:**
- NTILE(10) for performance deciles
- RANK() for top performers per wafer
- Price tier assignment (top 10% = premium, +$50/unit)

**SQL Pattern:**
```sql
WITH binning AS (
    SELECT device_id, test_value,
           NTILE(10) OVER (ORDER BY test_value DESC) as decile,
           RANK() OVER (PARTITION BY wafer_id ORDER BY test_value DESC) as wafer_rank
    FROM test_results WHERE test_name = 'Freq_Max'
)
SELECT device_id, decile,
       CASE 
           WHEN decile = 1 THEN 'Premium'
           WHEN decile <= 3 THEN 'Standard'
           ELSE 'Value'
       END as price_tier
FROM binning;
```

**Success Metrics:** Top 10% devices sell at 2× price → $20M additional revenue/year

---

#### **2. Test Result Trend Detection**
**Objective:** LAG/LEAD to detect process drift before yield impact  
**Deliverables:**
- 10-device moving average for smoothing
- Delta calculation (current vs previous)
- Alert if 5 consecutive devices show +0.05V drift

**SQL Pattern:**
```sql
WITH trends AS (
    SELECT device_id, test_value,
           LAG(test_value, 1) OVER (ORDER BY device_id) as prev_value,
           AVG(test_value) OVER (ORDER BY device_id ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) as ma_5
    FROM test_results WHERE test_name = 'Vdd_1.8V'
)
SELECT device_id, test_value - prev_value as delta
FROM trends
WHERE ABS(test_value - prev_value) > 0.05;
```

**Success Metrics:** Detect drift 50 devices earlier → prevent 500 false failures → $2M savings

---

#### **3. Test Dependency Optimization**
**Objective:** Recursive CTE to optimize test sequence and reduce test time  
**Deliverables:**
- Full test dependency graph
- Critical path identification
- Skip unnecessary tests if prerequisites fail

**SQL Pattern:**
```sql
WITH RECURSIVE critical_path AS (
    SELECT test_name, 0 as depth, test_name as path, 0 as total_time
    FROM tests WHERE prerequisite IS NULL
    UNION ALL
    SELECT t.test_name, cp.depth + 1, cp.path || ' → ' || t.test_name, cp.total_time + t.test_time_ms
    FROM tests t JOIN critical_path cp ON t.prerequisite = cp.test_name
)
SELECT path, total_time FROM critical_path
ORDER BY total_time DESC LIMIT 1;
```

**Success Metrics:** Reduce test time 18% by skipping dependent tests → $7M/year savings

---

#### **4. Spatial Correlation Analysis**
**Objective:** Window functions with spatial partitioning for wafer map analysis  
**Deliverables:**
- Average yield for 8 neighbors around each die
- Edge vs center die comparison
- Hot spot detection (clusters of failures)

**SQL Pattern:**
```sql
WITH spatial AS (
    SELECT device_id, die_x, die_y,
           AVG(CASE WHEN pass_fail = 'PASS' THEN 1.0 ELSE 0.0 END) OVER (
               PARTITION BY wafer_id
               ORDER BY die_x, die_y
               ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
           ) as neighbor_yield
    FROM test_results JOIN devices USING (device_id)
)
SELECT device_id, neighbor_yield
FROM spatial
WHERE neighbor_yield < 0.85;  -- Hot spot threshold
```

**Success Metrics:** Identify 85% of failures within 2mm of wafer edge → targeted scrape → $5M savings

---

### **General Data Analytics Projects**

#### **5. E-Commerce Customer Lifetime Value**
**Objective:** Window functions for customer segmentation and LTV analysis  
**Deliverables:** Cumulative revenue, RFM scores, churn prediction

**SQL Pattern:**
```sql
WITH customer_metrics AS (
    SELECT customer_id,
           SUM(order_total) OVER (PARTITION BY customer_id ORDER BY order_date) as cum_revenue,
           DATEDIFF(MAX(order_date) OVER (PARTITION BY customer_id), MIN(order_date) OVER (PARTITION BY customer_id)) as tenure_days
    FROM orders
)
SELECT customer_id, cum_revenue, 
       NTILE(4) OVER (ORDER BY cum_revenue DESC) as value_quartile
FROM customer_metrics;
```

---

#### **6. Financial Time Series Analysis**
**Objective:** LAG/LEAD for stock price trends and momentum indicators  
**Deliverables:** Moving averages, RSI, MACD, buy/sell signals

**SQL Pattern:**
```sql
SELECT date, price,
       AVG(price) OVER (ORDER BY date ROWS BETWEEN 49 PRECEDING AND CURRENT ROW) as ma_50,
       AVG(price) OVER (ORDER BY date ROWS BETWEEN 199 PRECEDING AND CURRENT ROW) as ma_200,
       price - LAG(price, 1) OVER (ORDER BY date) as daily_change
FROM stock_prices;
```

---

#### **7. Social Network Graph Analysis**
**Objective:** Recursive CTEs for influence analysis and community detection  
**Deliverables:** Follower depth, influencer ranking, viral content propagation

**SQL Pattern:**
```sql
WITH RECURSIVE followers AS (
    SELECT user_id, follower_id, 1 as depth
    FROM social_graph WHERE user_id = 'INFLUENCER_123'
    UNION ALL
    SELECT sg.user_id, sg.follower_id, f.depth + 1
    FROM social_graph sg JOIN followers f ON sg.user_id = f.follower_id
    WHERE f.depth < 5
)
SELECT COUNT(DISTINCT follower_id) as reach FROM followers;
```

---

#### **8. Healthcare Patient Readmission Patterns**
**Objective:** LAG to analyze readmission intervals and risk factors  
**Deliverables:** 30-day readmission rate, high-risk patient cohorts

**SQL Pattern:**
```sql
WITH readmissions AS (
    SELECT patient_id, admit_date,
           LAG(discharge_date) OVER (PARTITION BY patient_id ORDER BY admit_date) as prev_discharge,
           DATEDIFF(admit_date, LAG(discharge_date) OVER (PARTITION BY patient_id ORDER BY admit_date)) as days_since_discharge
    FROM admissions
)
SELECT patient_id, COUNT(*) as readmit_30d
FROM readmissions
WHERE days_since_discharge <= 30
GROUP BY patient_id;
```

---

**Next Steps:** Choose 1-2 projects, implement SQL queries, measure performance improvements

## 🎓 Part 6: Key Takeaways & Next Steps

### **What You've Learned**

✅ **Window Functions - Ranking**
- ROW_NUMBER, RANK, DENSE_RANK, NTILE for device binning
- PARTITION BY for per-wafer/per-lot analysis
- Performance tiers and Golden Unit selection

✅ **Window Functions - Sequential Analysis**
- LAG/LEAD for time-series trending
- Moving averages for noise reduction
- Drift detection and process monitoring

✅ **Recursive CTEs**
- Hierarchical data queries (test dependencies, org charts)
- Graph traversal (reachable nodes, shortest path)
- Bill of materials and genealogy tracking

✅ **Query Optimization**
- EXPLAIN ANALYZE for execution plans
- Indexes (single, composite, covering, partial)
- Query rewriting patterns
- 10-1000× speedup techniques

---

### **Advanced SQL Impact on Post-Silicon**

**Real-World Results:**
- **AMD**: Moving average smoothing → 60% noise reduction → clearer yield trends → $3M savings
- **NVIDIA**: Composite index (wafer_id, test_name) → 45s → 120ms (375× faster) → real-time dashboard
- **Qualcomm**: Recursive CTE for test dependencies → 18% test time reduction → $7M/year savings
- **Intel**: LAG for drift detection → 50 devices earlier warning → prevent 500 false failures → $2M savings

**Key Value Drivers:**
- ⚡ **Window Functions**: Complex analytics without GROUP BY collapse → richer insights
- 📊 **LAG/LEAD**: Sequential analysis 10× faster than subqueries
- 🔗 **Recursive CTEs**: Query hierarchies in single query vs N separate queries
- 🚀 **Optimization**: Proper indexing → 100-1000× speedup → production-ready systems

---

### **Window Functions vs Traditional SQL**

| **Task** | **Traditional SQL** | **Window Functions** |
|---|---|---|
| Device ranking | Subquery + COUNT | ROW_NUMBER() OVER |
| Moving average | Self-join N times | AVG() OVER (ROWS...) |
| Delta calculation | LEFT JOIN to self | LAG() OVER |
| Running total | Correlated subquery | SUM() OVER |
| Percentiles | Multiple queries | NTILE() OVER |

**Performance Impact:** Window functions are 5-50× faster than equivalent subqueries/joins.

---

### **Common Advanced SQL Patterns**

```sql
-- Pattern 1: Top N per group
SELECT * FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY wafer_id ORDER BY test_value DESC) as rn
    FROM test_results
) WHERE rn <= 10;

-- Pattern 2: Running total
SELECT device_id, test_value,
       SUM(test_value) OVER (ORDER BY device_id) as running_total
FROM test_results;

-- Pattern 3: Percent rank
SELECT device_id,
       PERCENT_RANK() OVER (ORDER BY test_value) as percentile
FROM test_results;

-- Pattern 4: Moving average with centering
SELECT device_id, test_value,
       AVG(test_value) OVER (
           ORDER BY device_id 
           ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING
       ) as ma_5_centered
FROM test_results;

-- Pattern 5: Find gaps in sequence
SELECT device_id, 
       LEAD(device_id) OVER (ORDER BY device_id) as next_device,
       LEAD(device_id) OVER (ORDER BY device_id) - device_id as gap
FROM devices
WHERE gap > 1;
```

---

### **Performance Optimization Checklist**

**Before Deploying to Production:**
- [ ] Index all foreign keys
- [ ] Index columns in WHERE clauses
- [ ] Composite index for multi-column filters (order matters!)
- [ ] Run EXPLAIN ANALYZE on slow queries
- [ ] Rewrite subqueries as JOINs or CTEs
- [ ] Use window functions instead of correlated subqueries
- [ ] Limit result set size (LIMIT, WHERE conditions)
- [ ] Monitor query performance over time (slowlog analysis)

**Index Strategy:**
- ✅ Index: WHERE, JOIN, ORDER BY columns
- ❌ Avoid: Over-indexing (>5 indexes per table)
- ✅ Composite: Multi-column filters (e.g., wafer_id + test_name)
- ✅ Partial: PostgreSQL partial indexes for subsets

---

### **Next Steps in Your Learning Journey**

**Immediate Next (Notebook 005: DSA Deep Dive):**
- Advanced data structures (trees, graphs, heaps)
- Graph algorithms for wafer map analysis
- Optimize query execution with better data structures

**Prerequisite Check:**
- ✅ Notebook 003: SQL Fundamentals
- ✅ Notebook 004: Advanced SQL (this notebook)

**Recommended Path:**
1. **005: DSA Deep Dive** - Algorithms for spatial analysis
2. **010: Linear Regression** - Apply optimized SQL for data loading
3. **091+: Data Engineering** - Distributed SQL (Spark SQL, Presto)

---

### **Resources for Further Learning**

**Window Functions:**
- PostgreSQL Window Functions documentation
- Mode Analytics SQL tutorial (window functions section)
- Use The Index Luke (windowing guide)

**Query Optimization:**
- PostgreSQL EXPLAIN documentation
- "SQL Performance Explained" book by Markus Winand
- Use The Index Luke website (comprehensive indexing guide)

**Recursive CTEs:**
- PostgreSQL WITH Queries documentation
- "Art of SQL" by Stéphane Faroult
- Graph database concepts (Neo4j)

**Practice:**
- LeetCode SQL (advanced problems)
- HackerRank SQL (window functions, CTEs)
- PostgreSQL exercises (pgexercises.com)

---

### **Final Thoughts**

Advanced SQL transforms data analysis from "slow batch processing" to "real-time interactive dashboards."

**Key Principles:**
- **Window functions** replace complex subqueries → simpler + faster
- **Recursive CTEs** handle hierarchies in single query → elegant + maintainable
- **Proper indexing** is non-negotiable for production → 100-1000× speedup
- **EXPLAIN everything** → understand query plans → optimize intelligently

**Production Mindset:**
- Every query should complete in <200ms for dashboards
- Index columns used in WHERE, JOIN, ORDER BY
- Use window functions for analytics (avoid correlated subqueries)
- Monitor slow query log and optimize top 10 slowest queries

**Next Action:** Open notebook 005 (DSA Deep Dive) and continue mastering algorithms! 🚀

---

**Notebook Complete!** 🎉

You now have advanced SQL skills for window functions, recursive CTEs, and query optimization. Apply these to build production-ready data pipelines and real-time analytics systems.